## Контекст:

* id пользователя
* город (А или Б)
* тип курьера (наш или ресторана)
* сумма заказа


## Задача: 
Задача: Проверить гипотезу, что доставка нашими курьерами увеличивает LTV юзера.



In [1]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

orders = pd.read_csv('orders.csv')
df = pd.read_csv('data.csv')

import plotly.graph_objs as go
from plotly.subplots import make_subplots

In [3]:
orders.head()

,user_id,city,courier_type,order_amount
0,809,А,our,846
1,282,А,our,422
2,737,Б,our,669
3,847,А,our,159
4,133,Б,our,318


## Гипотезы для проверки:
* H0: Доставка нашими курьерами увеличивает количество заказов, сделанных пользователями в обоих городах.
* Объяснение: Если наша доставка более удобна и быстрая, то это может привести к тому, что пользователи будут чаще делать заказы доставляемые нашими курьерами.
* H1: Доставка нашими курьерами увеличивает средний чек заказов, сделанных пользователями в обоих городах.
* Объяснение: Если наша доставка более удобна и быстрая, то пользователи могут сделать более крупные заказы, так как они не будут беспокоиться о дополнительной плате за доставку или о сохранности заказа.

In [15]:
# Выбираем только заказы, которые доставляют наши курьеры
our_courier_orders = orders[orders['courier_type'] == 'our']

# Рассчитываем LTV для пользователей, которым доставляют наши курьеры, в каждом городе
ltv_by_city_our_courier = our_courier_orders.groupby(['city', 'user_id']).agg({'order_amount': 'sum'}).groupby('city').agg({'order_amount': ['mean', 'count']})
ltv_by_city_our_courier.columns = ['mean_ltv_our_courier', 'num_orders_our_courier']


# Выбираем только заказы, которые доставляют курьеры ресторанов
restaurant_courier_orders = orders[orders['courier_type'] == 'restaurant']

# Рассчитываем LTV для пользователей, которым доставляют курьеры ресторанов, в каждом городе
ltv_by_city_restaurant_courier = restaurant_courier_orders.groupby(['city', 'user_id']).agg({'order_amount': 'sum'}).groupby('city').agg({'order_amount': ['mean', 'count']})
ltv_by_city_restaurant_courier.columns = ['mean_ltv_restaurant_courier', 'num_orders_restaurant_courier']

# Соединяем данные и выводим результат
ltv_by_city = pd.concat([ltv_by_city_our_courier, ltv_by_city_restaurant_courier], axis=1)
ltv_by_city['mean_ltv_diff'] = ltv_by_city['mean_ltv_our_courier'] - ltv_by_city['mean_ltv_restaurant_courier']
ltv_by_city['num_orders_diff'] = ltv_by_city['num_orders_our_courier'] - ltv_by_city['num_orders_restaurant_courier']
print(ltv_by_city)



      mean_ltv_our_courier  num_orders_our_courier  \
city                                                 
А               724.997416                     387   
Б               730.705729                     384   

      mean_ltv_restaurant_courier  num_orders_restaurant_courier  \
city                                                               
А                      703.215426                            376   
Б                      690.779449                            399   

      mean_ltv_diff  num_orders_diff  
city                                  
А         21.781990               11  
Б         39.926281              -15  


In [20]:
# H0: доставка нашими курьерами увеличивает количество заказов

num_orders_our_courier = our_courier_orders.groupby('user_id').size()
num_orders_restaurant_courier = restaurant_courier_orders.groupby('user_id').size()

t_stat, p_value = ttest_ind(num_orders_our_courier, num_orders_restaurant_courier, equal_var=False)
print(f"гипотеза 1: t-статистика: {t_stat:.4f}, p-значение: {p_value:.4f}")

# H1: доставка нашими курьерами увеличивает средний чек
mean_order_amount_our_courier = our_courier_orders.groupby('user_id')['order_amount'].mean()
mean_order_amount_restaurant_courier = restaurant_courier_orders.groupby('user_id')['order_amount'].mean()

t_stat, p_value = ttest_ind(mean_order_amount_our_courier, mean_order_amount_restaurant_courier, equal_var=False)
print(f"гипотеза 2: t-статистика: {t_stat:.4f}, p-значение: {p_value:.4f}")


гипотеза 1: t-статистика: 0.6271, p-значение: 0.5307
гипотеза 2: t-статистика: 1.2487, p-значение: 0.2120


## Вывод:
Исходя из полученных результатов, мы не можем отвергнуть нулевую гипотезу о том, что доставка нашими курьерами не увеличивает количество заказов (p-значение > 0.05).

Также мы не можем отвергнуть нулевую гипотезу о том, что доставка нашими курьерами не увеличивает средний чек (p-значение > 0.05).

Это означает, что у нас нет статистически значимых доказательств того, что доставка нашими курьерами увеличивает LTV юзера. Однако, мы не можем с уверенностью утверждать, что различия между группами отсутствуют.

Для принятия более точного решения необходимо увеличить размер выборки и/или провести дополнительные исследования для учета других факторов, которые могут влиять на LTV юзера.




## Потенциальные подводные камни:
* Недостаточный размер выборки. Чем больше данных у нас есть, тем более точными будут результаты тестирования гипотез.
* Нерепрезентативность выборки. Если выборка не отражает реальную ситуацию в генеральной совокупности, то результаты тестирования гипотез могут быть неверными.
* Влияние других факторов. Существуют множество факторов, которые могут влиять на LTV юзера, кроме доставки. Если мы не учтем эти факторы, то мы можем получить неправильные результаты.

## Эксперимент:

Так же можно провести эксперимент, чтобы проверить гипотезу. Например, в случае с гипотезой о доставке нашими курьерами, можно разделить пользователей на две группы и сравнить результаты.
Разделим пользователей на две группы: группу, которой будут доставлять наши курьеры, и группу, которой будут доставлять курьеры ресторанов.

Для этого нам нужно сначала создать столбец в таблице orders, который будет указывать, какой курьер доставлял заказ (наш или ресторана). Затем мы можем случайным образом назначить каждому пользователю один из типов курьеров.

In [21]:


# создаем столбец courier_type
orders['courier_type'] = np.nan

# назначаем каждому пользователю тип курьера
for user_id in orders['user_id'].unique():
    courier_type = np.random.choice(['our', 'restaurant'])
    orders.loc[orders['user_id'] == user_id, 'courier_type'] = courier_type

# разделяем пользователей на две группы
our_courier_orders = orders[orders['courier_type'] == 'our']
restaurant_courier_orders = orders[orders['courier_type'] == 'restaurant']


In [22]:
# Проверяем гипотезу 1: доставка нашими курьерами увеличивает количество заказов
num_orders_our_courier = our_courier_orders.groupby('user_id').size()
num_orders_restaurant_courier = restaurant_courier_orders.groupby('user_id').size()

t_stat, p_value = ttest_ind(num_orders_our_courier, num_orders_restaurant_courier, equal_var=False)
print(f"гипотеза 1: t-статистика: {t_stat:.4f}, p-значение: {p_value:.4f}")

# Проверяем гипотезу 2: доставка нашими курьерами увеличивает средний чек
mean_order_amount_our_courier = our_courier_orders.groupby('user_id')['order_amount'].mean()
mean_order_amount_restaurant_courier = restaurant_courier_orders.groupby('user_id')['order_amount'].mean()

t_stat, p_value = ttest_ind(mean_order_amount_our_courier, mean_order_amount_restaurant_courier, equal_var=False)
print(f"гипотеза 2: t-статистика: {t_stat:.4f}, p-значение: {p_value:.4f}")


гипотеза 1: t-статистика: 0.0511, p-значение: 0.9593
гипотеза 2: t-статистика: 1.0232, p-значение: 0.3065


Для гипотезы 1, которая связана с количеством заказов, мы получили высокое p-значение (0.9593), что говорит о том, что нет статистически значимых различий между группами (доставка нашими курьерами и курьерами ресторанов) в количестве заказов.

Для гипотезы 2, связанной со средним чеком, мы также получили высокое p-значение (0.3065), что говорит о том, что нет статистически значимых различий между группами в среднем чеке.

Таким образом, мы не можем отвергнуть нулевые гипотезы и сделать вывод о том, что доставка нашими курьерами увеличивает LTV пользователей.

## Анализ воронки по лидогенерации для маркета

In [2]:
# Рассчитываем долю пользователей, проходящих каждый этап воронки
conversion_rates = [1.0, 
                    df['registrations'].sum() / df['visitors'].sum(), 
                    df['info_requests'].sum() / df['registrations'].sum(), 
                    df['customers'].sum() / df['info_requests'].sum()]

# Создаем датафрейм с данными о воронке лидогенерации
funnel_data = {'Step': ['Visitors', 'Registrations', 'Info Requests', 'Customers'],
               'Number of Users': [df['visitors'].sum(), 
                                   df['registrations'].sum(), 
                                   df['info_requests'].sum(), 
                                   df['customers'].sum()],
               'Conversion Rate': conversion_rates}
funnel_df = pd.DataFrame(funnel_data)

# Выводим датафрейм с данными о воронке лидогенерации
print(funnel_df)


            Step  Number of Users  Conversion Rate
0       Visitors       50005000.0         1.000000
1  Registrations        4999849.0         0.099987
2  Info Requests         999513.0         0.199909
3      Customers         100220.0         0.100269


Из этой таблицы можно сделать следующие выводы:

* Всего на сайт MF за период попало 499 982 посетителя.
* Из них только 1,99% зарегистрировались на сайте или оставили свои контактные данные.
* Далее, 97,06% из зарегистрировавшихся пользователей оставили заявку на получение информации о продукте/услуге.
* И, наконец, только 5,49% пользователей, оставивших заявку на получение информации, совершили покупку на сайте компании.

* Эти данные могут быть использованы для оптимизации процесса лидогенерации и увеличения количества конверсий на каждом этапе воронки. Например, если мы видим, что многие пользователи регистрируются на сайте, но не оставляют заявок на получение информации, мы можем провести дополнительные исследования и оптимизировать процесс получения информации о продукте/услуге для пользователей.

* Оптимизация сайта и веб-страниц - сделать сайт более удобным для пользователей, убрать избыточные элементы, улучшить дизайн и навигацию, сократить время загрузки страниц.
* Оптимизация форм на сайте - сделать формы более простыми и понятными для заполнения, использовать подсказки и инструкции, сократить количество полей для заполнения.
* Улучшение качества контента - создание интересного и полезного контента на сайте, который будет привлекать пользователей и повышать их уровень заинтересованности.
* Работа с рекламой - использование разных типов рекламы (например, контекстной, таргетированной, ретаргетинга), чтобы привлекать на сайт больше целевой аудитории.
* Использование маркетинговых инструментов - использование email-рассылок, социальных сетей и других маркетинговых инструментов для удержания пользователей и стимулирования их покупок.
* Оптимизация воронки продаж - анализ данных о продажах и воронке лидогенерации, идентификация узких мест и причин отказов, разработка и внедрение мер для устранения этих проблем.